## Question 1 and 2
Copy A* algorithm from pathfinding.ipynb

## Question 3

In [56]:
class BoardConfiguration:
    def __init__(self, config: list[int]):
        self.config = config
    
    def neighbors(self) -> list:
        '''Return a list of the states 1 move away
        Don't wonder how, just have faith'''
        blank_index = self.config.index(None)
        result = []

        for offset in [-1, 1, -3, 3]:
            swap_index = blank_index + offset

            if (abs(offset) == 1 and swap_index // 3 != blank_index // 3):
                continue
            
            if swap_index in range(9):
                tmp = self.config.copy()
                tmp[swap_index], tmp[blank_index] = None, tmp[swap_index]
                result.append(tmp)

        return [BoardConfiguration(i) for i in result] 
    
    def __hash__(self) -> int:
        '''Return the hash value of the config
        Hash value := index of the blank space
        '''
        return self.config.index(None)
    
    def __lt__(self, other) -> bool:
        '''Compare on some arbitrary metric
        Needed for hashing
        '''
        return self.config.index(None) < other.config.index(None)
    
    def __eq__(self, other) -> bool:
        return self.config == other.config

    def __str__(self) -> str:
        result = "-" * 5 + "\n"
        for i in range(3):
            for j in range(3):
                value = self.config[i * 3 + j]
                result += (str(value) if value else " ") + " "
            result += "\n"
        result += "-" * 5
        return result

In [57]:
def backtrack(end: BoardConfiguration, prev: dict) -> list:
    '''Return backtracked path
    Path goes from `start` to `end`, constructed using parents from `prev`.
    `start` is any node such that `prev[start] = None`.
    '''
    path = []
    current = end
    while prev[current]:
        path.append(current)
        current = prev[current]
    return path[::-1]


In [58]:
def heuristic(current: BoardConfiguration, goal: BoardConfiguration) -> int:
    '''Return the minimum number of swaps necessary to reach goal state'''
    h = 0
    for i in range(9):
        x = goal.config.index(current.config[i])
        h += abs((i % 3) - (x % 3))     # col diff
        h += abs((i // 3) - (x // 3))   # row diff
    return h // 2 

In [59]:
from heapq import heappush, heappop
from collections import defaultdict

def a_star(start, goal) -> list[BoardConfiguration]:
    '''Return a list of BoardConfigurations from start to solved'''
    dist = defaultdict(lambda: float("inf"))  # initial distance estimates
    dist[start] = 0

    prev = defaultdict(lambda: None)
    prev[start] = None

    frontier = [(0, start)] # "frontier" to be explored 
    closed = set()

    while frontier:
        priority_current, current = heappop(frontier)
        
        if current == goal:
            break

        closed.add(current)

        for neighbor in current.neighbors():
            if neighbor in closed:
                continue

            alt_dist = dist[current] + 1 
            if alt_dist < dist[neighbor]:
                dist[neighbor] = alt_dist
                prev[neighbor] = current
                priority = alt_dist + heuristic(neighbor, goal)
                heappush(frontier, (priority, neighbor)) 

    return backtrack(current, prev)

In [60]:
goal_config = [2, 3, 1, None, 4, 5, 6, 7, 8]
start_config = [1, 2, 3, None, 4, 6, 7, 5, 8]

start = BoardConfiguration(start_config)
goal = BoardConfiguration(goal_config)

path = a_star(start, goal)
for b in path:
    print(b)

-----
1 2 3 
4   6 
7 5 8 
-----
-----
1 2 3 
4 6   
7 5 8 
-----
-----
1 2   
4 6 3 
7 5 8 
-----
-----
1   2 
4 6 3 
7 5 8 
-----
-----
  1 2 
4 6 3 
7 5 8 
-----
-----
4 1 2 
  6 3 
7 5 8 
-----
-----
4 1 2 
6   3 
7 5 8 
-----
-----
4   2 
6 1 3 
7 5 8 
-----
-----
4 2   
6 1 3 
7 5 8 
-----
-----
4 2 3 
6 1   
7 5 8 
-----
-----
4 2 3 
6   1 
7 5 8 
-----
-----
4 2 3 
6 5 1 
7   8 
-----
-----
4 2 3 
6 5 1 
  7 8 
-----
-----
4 2 3 
  5 1 
6 7 8 
-----
-----
  2 3 
4 5 1 
6 7 8 
-----
-----
2   3 
4 5 1 
6 7 8 
-----
-----
2 3   
4 5 1 
6 7 8 
-----
-----
2 3 1 
4 5   
6 7 8 
-----
-----
2 3 1 
4   5 
6 7 8 
-----
-----
2 3 1 
  4 5 
6 7 8 
-----
